# 📊 Ukrainian NPPs: Open Data Analysis 

This project explores open datasets on radioactive emissions and discharges from  
nuclear power plants (NPPs) in Ukraine. Using Python, Pandas, and Plotly, we  
analyze environmental indicators across several stations and visualize quarterly  
changes over time.

The goal is to better understand trends in emission levels and present the data  
in an accessible, interactive format. All data is retrieved from publicly  
available government sources.

🧪 **Technologies:** Python, Pandas, Plotly, Jupyter Notebook 

📈 **Focus:** Data analysis, API integration, interactive visualizations 

📂 **Data Source:** [Open data portal][1]

📦 **Dataset Page:** [Ecological and radiation situation in the area of ​​nuclear power plants][2]

🔌 **API docs:** [How to retrieve a dataset (API)?][3]

[1]: https://data.gov.ua/en/

[2]: https://data.gov.ua/en/dataset/4a9d3d56-bd95-4c3e-97e7-1cdc7bcbd445/resource/d55eebcf-4660-4919-96b3-4894be5a6cda

[3]: https://data.gov.ua/pages/aboutuser2#:~:text=%D0%AF%D0%BA%C2%A0%D0%B7%D0%B0%D0%B1%D1%80%D0%B0%D1%82%D0%B8%20%D0%BD%D0%B0%D0%B1%D1%96%D1%80%20%D0%B4%D0%B0%D0%BD%D0%B8%D1%85%20(API)%3F

## Work with API  

Create variables according to the API documentation for loading dataset metadata  
and exploring them. Takes dataset_id on a dataset page.

In [1]:
dataset_id = "4a9d3d56-bd95-4c3e-97e7-1cdc7bcbd445"

## Research dataset metadata

Download dataset metadata.

In [7]:
try:
    from src.client import DataGovUAClient
except ModuleNotFoundError:
    import sys
    from pathlib import Path


    project_root = Path().resolve().parent
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
else:
    from src.client import DataGovUAClient


sesion = DataGovUAClient(dataset_id)
sesion.fetch_metadata()


Metadata and resources is fetched.


You can reseach metadata with call `sesion.metadata`.

Add an identifier for the dataset passport and dataset. We see them in metadata.  
For more details, look carefully at the link

In [8]:
dataset_id = "d55eebcf-4660-4919-96b3-4894be5a6cda"
passport_id = "afa0c772-2554-4b9a-98b4-980e54b1e21a"

Next takes links on resources from metadata via id.

In [9]:
dataset_url = sesion.get_resource_url(resource_id=dataset_id)
passport_url = sesion.get_resource_url(resource_id=passport_id)

print(dataset_url)
print(passport_url)

https://data.gov.ua/dataset/c445c6ea-f0c3-4167-abb1-5afb4a0e5499/resource/d55eebcf-4660-4919-96b3-4894be5a6cda/download/nuclear_safety_q4_2024.xlsx
https://data.gov.ua/dataset/c445c6ea-f0c3-4167-abb1-5afb4a0e5499/resource/afa0c772-2554-4b9a-98b4-980e54b1e21a/download/pasport-naboru-danikh.xlsx


## Dataset passport overview

Now we have two different dataset: it's passport and dataset with annual  
observations. First of all, let"s see at a pasport  of dataset. Let's put  
it in a tabular form.

In [10]:
passport_df = sesion.load_dataframe(resource_url=passport_url)

# Customizes table styles.
columns_rename = list(passport_df.columns[:-1]) + ["Примітка"]
passport_df.columns = columns_rename

(
    passport_df.fillna("").style.set_table_styles(
        [
            {"selector": "th", "props": [("text-align", "center")]},
            {
                "selector": "td",
                "props": [
                    ("white-space", "pre-wrap"),
                    ("text-align", "left"),
                    ("width", "300px"),
                ],
            },
        ]
    )
)

,Назва набору,Екологічна та радіаційна обстановка в зоні розташування атомних електростанцій,Примітка
0,Формати файлів,"xlsx, csv",
1,Шаблон назв файлів,nuclear_safety_QN_РРРР,
2,Ключові слова,"енергоатом, викид, викиди, скид, скиди, радіоактивність, цезій, кобальт, аес, атомна станція, атомні станції",
3,Періодичність оприлюднення,"Щоквартально, до 25 числа місяця, наступного за звітнім періодом",
4,Додаткові уточнення,"У випадку відсутності інформації або наявності значень нижчих за мінімальну активність, що може бути виміряною, залишати комірку пустою",
5,,,
6,Структура набору:,,
7,Назва поля,Переклад на українську,Опис
8,year,рік,Формат РРРР
9,quarter,квартал,"Число від 1 до 4, де ""1"" означає період січень-березень ""2"" - квітень-червень ""3"" - липень-вересень ""4"" - жовтень-грудень"


For analysis and detailing, we will select indicators under the following 
indices metrics:

In [ ]:
metrics = [
    "iodine_ radionuclides",
    "iodine_ radionuclides_index",
    "stable_radionuclides",
    "cs_137_emission",
    "co_60_ emission",
    "cs_137_dump",
    "co_60_dump",
    "index_radioactive_releas",
    "index_dump",
]

['iodine_radionuclides',
 'iodine_radionuclides_index',
 'stable_radionuclides',
 'cs_137_emission',
 'co_60_emission',
 'cs_137_dump',
 'co_60_dump',
 'index_radioactive_releas',
 'index_dump']

## Dataset acquisition and analysis

First, let's download the dataset from the link.

In [12]:
dataset_df = sesion.load_dataframe(resource_url=dataset_url)
dataset_df.head()

,year,quarter,station,irg,irg_index,iodine_ radionuclides,iodine_ radionuclides_index,stable_radionuclides,stable_ radionuclides_index,cs_137_emission,co_60_ emission,cs_137_dump,co_60_dump,volume,index_radioactive_releas,index_dump
0,2018,1,ЗАЕС,89.0,0.13,260.0,"<0,01",650.0,0.03,1980.0,1020.0,4330.0,3670.0,833000.0,0.149,0.33
1,2018,1,РАЕС,105.0,0.16,147.0,"<0,01",269.0,0.07,587.0,165.0,4800.0,620.0,2220000.0,0.780,0.096
2,2018,1,ЮУАЕС,45.0,0.10,76.0,"<0,01",116.0,0.02,136.0,373.0,390.0,370.0,14600.0,0.136,0.284
3,2018,1,ХАЕС,31.0,0.07,26.8,"<0,01",37.5,"<0,01",29.4,13.8,380.0,NaN,22070.0,0.110,0.03
4,2018,2,ЗАЕС,84.0,0.12,262.0,"<0,01",640.0,0.03,453.0,1003.0,4627.0,3432.0,812667.0,0.115,0.91


### Format data
We will format the quarter and year data as "DD.MM.YYYY".  
This will be needed for visualization.

In [ ]:
from src.utils import get_date


dataset_df["date"] = dataset_df.apply(
    lambda row: get_date(year=row["year"], quarter=row["quarter"]), axis=1
)
dataset_df[["year", "quarter", "date"]].head()

,year,quarter,date
0,2018,1,31.03.2018
1,2018,1,31.03.2018
2,2018,1,31.03.2018
3,2018,1,31.03.2018
4,2018,2,30.06.2018


Let's list all nuclear power plants.

In [16]:
stations = dataset_df["station"].unique()
tuple(stations)

('ЗАЕС', 'РАЕС', 'ЮУАЕС', 'ХАЕС', 'ПАЕС')

In April 2022, Energoatom, by its order, approved the change of the name of the  
NPP from “South Ukrainian Nuclear Power Plant” to “South Ukrainian Nuclear Power  
Plant”. In accordance with the Resolution of the Cabinet of Ministers No. 1061  
of September 27, 2022, the government finally approved the renaming of the NPP  
from “South Ukrainian Nuclear Power Plant” to “South Ukrainian” 
[1](https://epravda.com.ua/news/2022/09/29/692034/).

Will make station names more readable.

In [20]:
station_map = {
    "ЗАЕС": "Zaporizhzhia NPP",
    "РАЕС": "Rivne NPP",
    "ПАЕС": "South Ukrainian NPP",
    "ХАЕС": "Khmelnytskyi NPP",
    "ЮУАЕС": "South Ukrainian NPP",
}

dataset_df["station_en"] = dataset_df["station"].map(station_map)
dataset_df[["station", "station_en"]].head()

,station,station_en
0,ЗАЕС,Zaporizhzhia NPP
1,РАЕС,Rivne NPP
2,ЮУАЕС,South Ukrainian NPP
3,ХАЕС,Khmelnytskyi NPP
4,ЗАЕС,Zaporizhzhia NPP


As you can see in the result, the replacement was successful.

## Dataset visualization 

In [ ]:
import plotly.graph_objects as go

stations_en_list = sorted(set(station_map.values()))
index_radioactive_max = dataset_df["index_radioactive_releas"].max()

fig = go.Figure()

# Create a separate graph with data for each station.
traces = []
for i, station in enumerate(stations_en_list):
    df_station = dataset_df[dataset_df["station_en"] == station]

    trace = go.Scatter(
        x=df_station["date"],
        y=df_station["index_radioactive_releas"],
        name=station,
        visible=True,
    )
    traces.append(trace)
    fig.add_trace(trace)

# Customize buttons for selecting stations.
buttons = []

# Button for selecting all stations.
buttons.append(
    dict(
        label="All",
        method="update",
        args=[
            {"visible": [True] * len(traces)},
            {"title": "Radioactive Release — All Station"},
        ],
    )
)

# Buttons for selecting separately station.
for i, station in enumerate(stations_en_list):
    visibility = [False] * len(traces)
    visibility[i] = True
    buttons.append(
        dict(
            label=station,
            method="update",
            args=[
                {"visible": visibility},
                {"title": f"Radioactive Release — {station}"},
            ],
        )
    )

# Add a menu.
fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            active=0,
            buttons=buttons,
            xanchor="right",
            x=1,
            y=1.2,
        )
    ],
    title=f"Radioactive Release — {station}",
    xaxis_title="Date",
    xaxis=dict(tickangle=-45, showgrid=True, showline=True, domain=[0, 0.94]),
    yaxis_title="Release Index",
    yaxis=dict(range=[0, index_radioactive_max + 1]),
    template="plotly_dark",
    width=800,
    height=600,
    legend=dict(
        title="Station",
        orientation="v",  # вертикальне розміщення
        x=1.02,  # праворуч від графіку
        y=1,  # зверху
        xanchor="left",  # легенда притиснута до лівого краю x=1.02
        yanchor="top",
        bgcolor="rgba(0,0,0,0)",  # прозорий фон
        borderwidth=1,
        font=dict(
            size=12,
            color="white",  # або "black" залежно від теми
        ),
        itemwidth=40,  # фіксує ширину кожного елементу (потрібно для стабільності)
    ),
    margin=dict(r=150),
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

stations_en_list = sorted(dataset_df["station_en"].unique())


# Максимальні значення для масштабування по y
max_values = {
    metric: round(dataset_df[metric].max() + 1)
    if pd.notnull(dataset_df[metric].max())
    else 1
    for metric in metrics
}

fig = go.Figure()
traces = []

# Створення всіх графіків
for metric in metrics:
    for station in stations_en_list:
        df_station = dataset_df[dataset_df["station_en"] == station]
        trace = go.Scatter(
            x=df_station["date"],
            y=df_station[metric],
            name=station,
            visible=(metric == "index_radioactive_releas"),
            legendgroup=station,
        )
        fig.add_trace(trace)
        traces.append((metric, station, trace))

# Кнопки для перемикання метрик
buttons = []
for metric in metrics:
    visibility = [(m == metric) for (m, _, _) in traces]
    buttons.append(
        dict(
            label=metric,
            method="update",
            args=[
                {"visible": visibility},
                {
                    "title": f"Metric: {metric}",
                    "yaxis": {"range": [0, max_values[metric]]},
                    "yaxis_title": metric.replace("_", " ").title(),
                },
            ],
        )
    )

# Оновлення макету
fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            active=metrics.index("index_radioactive_releas"),
            buttons=buttons,
            xanchor="right",
            x=1,
            y=1.2,
        )
    ],
    title="Metric: index_radioactive_releas",
    xaxis_title="Date",
    xaxis=dict(tickangle=-45, showgrid=True, showline=True, domain=[0, 0.94]),
    yaxis_title="Release Index",
    yaxis=dict(
        range=[0, max_values["index_radioactive_releas"]],
    ),
    template="plotly_dark",
    width=1000,
    height=600,
    legend=dict(
        title="Station",
        orientation="v",
        x=1.02,
        y=1,
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(0,0,0,0)",
        borderwidth=1,
        font=dict(size=12, color="white"),
        itemwidth=40,
    ),
    margin=dict(r=200),
)

fig.show()


### Висновки
1. В Україні є чотири Атомні електростанції.
2. В другій половині 2021 року на **ЮУАЕС** сталася ситуація, що причинила підвищення сумарних викидів в атмосферу більше ніж у 30 разів.
3. В першому кварталі 2022 року ([22 січня 2022 року о 14:08](https://uk.wikipedia.org/wiki/%D0%9F%D1%96%D0%B2%D0%B4%D0%B5%D0%BD%D0%BD%D0%BE%D1%83%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D1%81%D1%8C%D0%BA%D0%B0_%D0%90%D0%95%D0%A1)) **ЮУАЕС** припинила роботу, що співпадає з відкритими даними на [Вікіпедії](https://uk.wikipedia.org/wiki/%D0%9F%D1%96%D0%B2%D0%B4%D0%B5%D0%BD%D0%BD%D0%BE%D1%83%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D1%81%D1%8C%D0%BA%D0%B0_%D0%90%D0%95%D0%A1). Після відключення дані про викиди станції не надходили.
4. З четвертого кварталу 2022 року дані про викиди **ЗАЕС** не надходять через війсьові злочини рф.
5. На сьогоднішній день у штатному режимі працюють станції **РАЕС** та **ХАЕС**.